<a href="https://colab.research.google.com/github/faizankshaikh/SocialNeuro/blob/main/SocialNeuro/envs/cfg/config_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
arr = []

weather_types = [0.2, 0.3, 0.4]
weather_types_coop = [0.36, 0.51, 0.64]

def prob_payoff(self_action, other_action, self_life_points, other_life_points, weather_type_idx):
    if self_life_points != 0:
        if other_life_points != 0:
            if self_action == 1 and other_action == 1:
                return weather_types_coop[weather_type_idx]
            elif self_action == 1 and other_action == 0:
                return weather_types[weather_type_idx]
            elif self_action == 0 and other_action == 1:
                return 1
            else:
                return 0
        else:
            if self_action == 0:
                return 0
            else:
                return weather_types[weather_type_idx]
    else:
        return 0


for days in range(0, 3):
    for p1_lp in range(0, 4):
        for p2_lp in range(0, 4):
            for p1_a in range(0, 2):
                for p2_a in range(0, 2):
                    for w_idx, w in enumerate(weather_types):
                        p1_p = prob_payoff(p1_a, p2_a, p1_lp, p2_lp, w_idx)
                        p2_p = prob_payoff(p2_a, p1_a, p2_lp, p1_lp, w_idx)

                        if p1_lp > 0 and p2_lp > 0 and days > 0:
                            is_init_val = 1
                        else:
                            is_init_val = 0

                        arr.append([is_init_val, days, p1_lp, p2_lp, p1_a, p2_a, w_idx, p1_p, p2_p])

data = pd.DataFrame(arr, columns=["is_init_val", "days_left", "player1_life_points", "player2_life_points", "player1_action", "player2_action", "weather_type", "player1_prob_payoff", "player2_prob_payoff"])
data.to_csv("comp_v0_game_initialization.csv", index=False)

In [4]:
gain = 1
cost = -1
gain_dead = 0

def get_payoffs(player1_alive, player2_alive, player1_action, player2_action, player1_possible_outcome, player2_possible_outcome):

    if player1_alive and player2_alive:
        if player1_action == player2_action == 1:
            if player1_possible_outcome and player2_possible_outcome:
                player1_payoff, player2_payoff = gain, gain
            else:
                player1_payoff, player2_payoff = cost, cost

        elif player1_action == 1 and player2_action == 0:
            # player1 forage success, player2 attack success
            if player1_possible_outcome and player2_possible_outcome:
                player1_payoff, player2_payoff = gain + cost, gain
            # player1 forage success, player2 attack failure
            elif player1_possible_outcome and not player2_possible_outcome:
                player1_payoff, player2_payoff = gain, cost
            # player1 forage failure, player2 attack success
            elif not player1_possible_outcome and player2_possible_outcome:
                player1_payoff, player2_payoff = cost + cost, gain
            # player1 forage failure, player2 attack failure
            else:
                player1_payoff, player2_payoff = cost, cost
        elif player1_action == 0 and player2_action == 1:
            # player1 attack success, player2 forage success
            if player1_possible_outcome and player2_possible_outcome:
                player1_payoff, player2_payoff = gain, gain + cost
            # player1 attack success, player2 forage failure
            elif player1_possible_outcome and not player2_possible_outcome:
                player1_payoff, player2_payoff = gain, cost + cost
            # player1 attack failure, player2 forage success
            elif not player1_possible_outcome and player2_possible_outcome:
                player1_payoff, player2_payoff = cost, gain
            # player1 attack failure, player2 forage failure
            else:
                player1_payoff, player2_payoff = cost, cost
        else:
            # both players attack
            player1_payoff, player2_payoff = cost, cost
    elif player1_alive and not player2_alive:
        if player1_action == 1:
            if player1_possible_outcome:
                player1_payoff, player2_payoff = gain, gain_dead
            else:
                player1_payoff, player2_payoff = cost, gain_dead
        else:
            player1_payoff, player2_payoff = cost, gain_dead
    elif not player1_alive and player2_alive:
        if player2_action == 1:
            if player2_possible_outcome:
                player1_payoff, player2_payoff = gain_dead, gain
            else:
                player1_payoff, player2_payoff = gain_dead, cost
        else:
            player1_payoff, player2_payoff = gain_dead, cost
    else:
        player1_payoff, player2_payoff = gain_dead, gain_dead

    return player1_payoff, player2_payoff


arr = []

for p1_lp in range(0, 2):
    for p2_lp in range(0, 2):
        for p1_a in range(0, 2):
            for p2_a in range(0, 2):
                for p1_o in range(0, 2):
                    for p2_o in range(0, 2):
                        p1_p, p2_p = get_payoffs(p1_lp, p2_lp, p1_a, p2_a, p1_o, p2_o)

                        if p1_lp == 0: p1_a = 2
                        if p2_lp == 0: p2_a = 2

                        arr.append([p1_lp, p2_lp, p1_a, p2_a, p1_o, p2_o, p1_p, p2_p])


data = pd.DataFrame(arr, columns=["player1_alive", "player2_alive", "player1_action", "player2_action",
                                  "player1_possible_outcome", "player2_possible_outcome", "player1_payoff", "player2_payoff"])
data.to_csv("comp_v0_game_mechanics.csv", index=False)